In [12]:
!pip install kafka-python

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 246 kB 17.0 MB/s 


In [ ]:
!pip install s3fs

In [8]:
#%%bash
# instructions from https://docs.aws.amazon.com/cli/latest/userguide/getting-started-install.html
!curl "https://awscli.amazonaws.com/awscli-exe-linux-x86_64.zip" -o "awscliv2.zip"
!unzip -q awscliv2.zip
!sudo ./aws/install

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 45.9M  100 45.9M    0     0   399M      0 --:--:-- --:--:-- --:--:--  399M
replace aws/install? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
You can now run: /usr/local/bin/aws --version


In [9]:
!aws --version

aws-cli/2.9.8 Python/3.9.11 Linux/5.10.133+ exe/x86_64.ubuntu.18 prompt/off


In [10]:
!aws configure

AWS Access Key ID [None]: 


In [15]:
import pandas as pd
from kafka import KafkaProducer, KafkaConsumer
from time import sleep
import json
from json import dumps
from s3fs import S3FileSystem

In [16]:
producer = KafkaProducer(bootstrap_servers = ['13.126.222.247:9092'],
                         value_serializer = lambda x:
                         dumps(x).encode('utf-8'))

In [22]:
df1 = pd.read_csv("/content/sp500_companies.csv")
df3 = pd.read_csv("/content/sp500_stocks.csv")


In [ ]:
df1.head()

,Exchange,Symbol,Shortname,Longname,Sector,Industry,Currentprice,Marketcap,Ebitda,Revenuegrowth,City,State,Country,Fulltimeemployees,Longbusinesssummary,Weight
0,NYQ,APH,Amphenol Corporation,Amphenol Corporation,Technology,Electronic Components,81.97,48779935744,2.938300e+09,0.169,Wallingford,CT,United States,90000,"Amphenol Corporation, together with its subsid...",0.026877
1,NYQ,ROP,"Roper Technologies, Inc.","Roper Technologies, Inc.",Industrials,Specialty Industrial Machinery,443.85,47071182848,2.348600e+09,0.096,Sarasota,FL,United States,19300,"Roper Technologies, Inc. designs and develops ...",0.025935
2,NYQ,AJG,Arthur J. Gallagher & Co.,Arthur J. Gallagher & Co.,Financial Services,Insurance Brokers,193.74,40848142336,2.309800e+09,-0.050,Rolling Meadows,IL,United States,32061,"Arthur J. Gallagher & Co., together with its s...",0.022506
3,NYQ,TT,Trane Technologies plc,Trane Technologies plc,Industrials,Building Products & Equipment,176.54,40658399232,2.544900e+09,0.175,Swords,NaN,Ireland,37000,"Trane Technologies plc, together with its subs...",0.022402
4,NYQ,IQV,"IQVIA Holdings, Inc.",IQVIA Holdings Inc.,Healthcare,Diagnostics & Research,213.14,39588622336,2.760000e+09,0.050,Durham,NC,United States,85000,IQVIA Holdings Inc. provides advanced analytic...,0.021812


In [ ]:
df3.head()

,Date,Symbol,Adj Close,Close,High,Low,Open,Volume
0,2010-01-04,MMM,59.318886,83.019997,83.449997,82.669998,83.089996,3043700.0
1,2010-01-05,MMM,58.947342,82.500000,83.230003,81.699997,82.800003,2847000.0
2,2010-01-06,MMM,59.783295,83.669998,84.599998,83.510002,83.879997,5268500.0
3,2010-01-07,MMM,59.826176,83.730003,83.760002,82.120003,83.320000,4470100.0
4,2010-01-08,MMM,60.247749,84.320000,84.320000,83.300003,83.690002,3405800.0


In [26]:
while True:
  dict_stock=df1.sample(1).to_dict(orient="records")[0]
  producer.send('sp500_comp', value=dict_stock)
  sleep(2)

KeyboardInterrupt: ignored

In [25]:
while True:
  dict_stock2=df3.sample(1).to_dict(orient="records")[0]
  producer.send('sp500_stocks', value=dict_stock2)
  sleep(2)

KeyboardInterrupt: ignored

In [ ]:
producer.flush()
